In [2]:
import konlpy
import nltk
import json
import urllib.request
import requests
import re
from googletrans import Translator

In [51]:
print("보그체로 변환하고 싶은 뉴스 기사(한국어) url 입력하시오")
Korean_Text_b = '안녕하세요, 만나서 반갑습니다. 사이좋게 지냅시다'
print(Korean_Text_b)

보그체로 변환하고 싶은 뉴스 기사(한국어) url 입력하시오
안녕하세요, 만나서 반갑습니다. 사이좋게 지냅시다


In [52]:
Korean_Text = Korean_Text_b.split(sep=' ')
Korean_Text

['안녕하세요,', '만나서', '반갑습니다.', '사이좋게', '지냅시다']

In [53]:
Toknize_Okt = konlpy.tag.Okt()
Toknize_Hannanum = konlpy.tag.Hannanum()
Toknize_Kkma = konlpy.tag.Kkma()

Toknized_Korean_Okt = []
Toknized_Korean_Hannanum = []
Toknized_Korean_Kkma = []


for i in range(len(Korean_Text)):
    Toknized_Korean_Okt.append(Toknize_Okt.pos(Korean_Text[i]))
    Toknized_Korean_Hannanum.append(Toknize_Hannanum.pos(Korean_Text[i]))
    Toknized_Korean_Kkma.append(Toknize_Kkma.pos(Korean_Text[i]))

    for j in range(len(Toknized_Korean_Okt[i])):
        Toknized_Korean_Okt[i][j] = list(Toknized_Korean_Okt[i][j])

    for j in range(len(Toknized_Korean_Hannanum[i])):
        Toknized_Korean_Hannanum[i][j] = list(Toknized_Korean_Hannanum[i][j])

    for j in range(len(Toknized_Korean_Kkma[i])):
        Toknized_Korean_Kkma[i][j] = list(Toknized_Korean_Kkma[i][j])


In [54]:
Toknized_Korean_Okt

[[['안녕하세요', 'Adjective'], [',', 'Punctuation']],
 [['만나서', 'Verb']],
 [['반갑습니다', 'Adjective'], ['.', 'Punctuation']],
 [['사이좋게', 'Adjective']],
 [['지냅시다', 'Verb']]]

In [55]:
Toknized_Korean_Hannanum

[[['안녕하세요', 'N'], [',', 'S']],
 [['만나', 'N'], ['서', 'J']],
 [['반갑', 'P'], ['습니다', 'E'], ['.', 'S']],
 [['사이좋', 'P'], ['게', 'E']],
 [['지내', 'P'], ['ㅂ시다', 'E']]]

In [56]:
Toknized_Korean_Kkma

[[['안녕', 'NNG'], ['하세', 'NNG'], ['요', 'JX'], [',', 'SP']],
 [['만나', 'VV'], ['서', 'ECD']],
 [['반갑', 'VA'], ['습니다', 'EFN'], ['.', 'SF']],
 [['사이', 'NNG'], ['좋', 'VA'], ['게', 'ECD']],
 [['지내', 'VV'], ['ㅂ시다', 'EFA']]]

In [57]:
def ends_with_jongsung(letter): # 받침 유무 확인하기 받침이 있으면 True, 없으면 False 반환
    m = re.search(r"[가-힣|ㄴ]?", letter)
    if m:
        n = m.group()[-1]
        return (ord(n)-ord("가")) % 28 > 0 # 나머지가 있으면 받침 있음. 반환값 생김.
    else:
        return None

In [58]:
ends_with_jongsung("ㄴ")

True

In [59]:
def josa_change(letter, letter2): # 주격 조사 알맞게 change
    if letter2 == "을" or letter2 == "를":
        josa = "을" if ends_with_jongsung(letter) else "를" # 앞에 받침 있는 경우, 뒤에는 받침이 없는 경우
    elif letter2 == "이" or letter2 == "가":
        josa = "이" if ends_with_jongsung(letter) else "가"
    elif letter2 == "과" or letter2 == "와":
        josa = "과" if ends_with_jongsung(letter) else "와"
    elif letter2 == "은" or letter2 == "는":
        josa = "은" if ends_with_jongsung(letter) else "는"

    return josa

In [60]:
def guanhyeongsa_eomi_change(letter, letter2): # 관형사 어미 알맞게 change
    '''
    if letter2 == "ㅇ"
    guanhyeonsa_eomi = "하는" if ends_with_jongsung(letter) else "는" # 받침 있고, 받침 없고
    guanhyeonsa_eomi = "한"  if ends_with_jongsung(letter) else ""

    return guanhyeonsa_eomi
    '''
    pass

In [61]:
## Kkma 사용하기
'''
valid_grammer_Okt = ['Noun', 'Verb', 'Adjective', 'Determiner', 'Adverb', 'Conjunction', 'Exclamation']
valid_grammer_Hannanum = ['N', 'P', 'M', 'I']
valid_grammer_Kkma = ['NNP', 'NNG', 'NNB', 'NNBC', 'NR', 'NP', 'VV', 'VA', 'VXV', 'VXA', 'VCP', 'VCP', 'VCN', 'MDN', 'MDT', 'MAG', 'MAC', 'IC']
non_valid_grammer_eomi_Kkma = ['EFN', 'EFQ', 'EFO', 'EFA', 'EFI', 'EFR']
'''

valid_grammer_Okt = ['Noun']
valid_grammer_Hannanum = ['N']
valid_grammer_Kkma = ['NNP', 'NNG', 'NNB', 'NNBC', 'NR', 'NP', 'IC']
non_valid_grammer_eomi_Kkma = ['EFN', 'EFQ', 'EFO', 'EFA', 'EFI', 'EFR']


stopwords =['있']


client_id = "DItIw9wyPPPRDPx4Hy46"
client_secret = "L6ZEleX9Mp"
url = "https://openapi.naver.com/v1/papago/n2mt"
url2 = "https://transliterator.herokuapp.com/"
sess = requests.Session()

vogulized_sentence = ''

for word in range(len(Toknized_Korean_Hannanum)):
    for w_divided in range(len(Toknized_Korean_Hannanum[word])):
            if Toknized_Korean_Hannanum[word][w_divided][1] in valid_grammer_Hannanum and Toknized_Korean_Hannanum[word][w_divided][0] not in stopwords:
                print("번역 가능한 품사 단어 : " + Toknized_Korean_Hannanum[word][w_divided][0])
                Toknized_Text_English = urllib.parse.quote(Toknized_Korean_Hannanum[word][w_divided][0])
                data = "source=ko&target=en&text=" + Toknized_Text_English
                request = urllib.request.Request(url)
                request.add_header("X-Naver-Client-Id", client_id)
                request.add_header("X-Naver-Client-Secret", client_secret)
                response = urllib.request.urlopen(request, data=data.encode("utf-8"))
                rescode = response.getcode()

                if rescode==200:
                    response_body = response.read()
                    json_word = json.loads(response_body.decode("utf-8"))
                    result = json_word['message']['result']['translatedText']
                    payload = {'input': result}
                    res = sess.post(url2, data=payload)
                    konglish = res.json()['output'].replace('?','')
                    Toknized_Korean_Kkma[word][w_divided][0] = konglish
                    print(konglish)
                    vogulized_sentence += konglish

                else:
                    print("Error")

            else:
                print("번역 불가능한 품사 단어 : " + Toknized_Korean_Kkma[word][w_divided][0])
                if Toknized_Korean_Kkma[word][w_divided][1] == 'JKS':
                    Toknized_Korean_Kkma[word][w_divided][0] = josa_change(Toknized_Korean_Kkma[word][w_divided-1][0][-1], Toknized_Korean_Kkma[word][w_divided][0])

                if Toknized_Korean_Kkma[word][w_divided][1] == 'ETD' or Toknized_Korean_Kkma[word][w_divided][1] =='EFN':
                    if Toknized_Korean_Kkma[word][w_divided][0] == "ㄴ":
                        Toknized_Korean_Kkma[word][w_divided][0] == "한"
                    elif Toknized_Korean_Kkma[word][w_divided][0] == "ㄴ다":
                        Toknized_Korean_Kkma[word][w_divided][0] == "한다"



                vogulized_sentence += str(Toknized_Korean_Kkma[word][w_divided][0])

    vogulized_sentence += ' '


print(vogulized_sentence)
print(Korean_Text_b)

번역 가능한 품사 단어 : 안녕하세요
헬로
번역 불가능한 품사 단어 : 하세
번역 가능한 품사 단어 : 만나
레츠미트
번역 불가능한 품사 단어 : 서
번역 불가능한 품사 단어 : 반갑
번역 불가능한 품사 단어 : 습니다
번역 불가능한 품사 단어 : .
번역 불가능한 품사 단어 : 사이
번역 불가능한 품사 단어 : 좋
번역 불가능한 품사 단어 : 지내
번역 불가능한 품사 단어 : ㅂ시다
헬로하세 레츠미트서 반갑습니다. 사이좋 지내ㅂ시다 
안녕하세요, 만나서 반갑습니다. 사이좋게 지냅시다


In [62]:
Toknized_Korean_Kkma # 앞에 받침 있으면 하려, 없으면 려

[[['헬로', 'NNG'], ['하세', 'NNG'], ['요', 'JX'], [',', 'SP']],
 [['레츠미트', 'VV'], ['서', 'ECD']],
 [['반갑', 'VA'], ['습니다', 'EFN'], ['.', 'SF']],
 [['사이', 'NNG'], ['좋', 'VA'], ['게', 'ECD']],
 [['지내', 'VV'], ['ㅂ시다', 'EFA']]]

In [63]:
translator = Translator(service_urls=[
      'translate.google.co.kr',
    ])
result = translator.translate(Korean_Text_b, src='ko', dest='en')
print(result.text)

Hi, nice to meet you.Let's get along


In [64]:
Korean_Text_b

'안녕하세요, 만나서 반갑습니다. 사이좋게 지냅시다'

In [66]:
'''
valid_grammer_Okt = ['Noun', 'Verb', 'Adjective', 'Determiner', 'Adverb', 'Conjunction', 'Exclamation']
valid_grammer_Hannanum = ['N', 'P', 'M', 'I']
valid_grammer_Kkma = ['NNP', 'NNG', 'NNB', 'NNBC', 'NR', 'NP', 'VV', 'VA', 'VXV', 'VXA', 'VCP', 'VCP', 'VCN', 'MDN', 'MDT', 'MAG', 'MAC', 'IC']
non_valid_grammer_eomi_Kkma = ['EFN', 'EFQ', 'EFO', 'EFA', 'EFI', 'EFR']
'''

valid_grammer_Okt = ['Noun']
valid_grammer_Hannanum = ['N']
valid_grammer_Kkma = ['NNP', 'NNG', 'NNB', 'NNBC', 'NR', 'NP', 'IC']
non_valid_grammer_eomi_Kkma = ['EFN', 'EFQ', 'EFO', 'EFA', 'EFI', 'EFR']


stopwords =['있']


translator = Translator(service_urls=['translate.google.co.kr'])
url2 = "https://transliterator.herokuapp.com/"
sess = requests.Session()
vogulized_sentence = ''

for word in range(len(Toknized_Korean_Hannanum)):
    for w_divided in range(len(Toknized_Korean_Hannanum[word])):
            if Toknized_Korean_Hannanum[word][w_divided][1] in valid_grammer_Hannanum and Toknized_Korean_Hannanum[word][w_divided][0] not in stopwords:
                print("번역 가능한 품사 단어 : " + Toknized_Korean_Hannanum[word][w_divided][0])

                konglish = translator.translate(Toknized_Korean_Hannanum[word][w_divided][0], src='ko', dest='en').text
                payload = {'input': konglish}
                res = sess.post(url2, data=payload)
                konglish = res.json()['output'].replace('?','')
                Toknized_Korean_Kkma[word][w_divided][0] = konglish
                print(konglish)

                vogulized_sentence += konglish

            else:
                print("번역 불가능한 품사 단어 : " + Toknized_Korean_Kkma[word][w_divided][0])
                if Toknized_Korean_Kkma[word][w_divided][1] == 'JKS':
                    Toknized_Korean_Kkma[word][w_divided][0] = josa_change(Toknized_Korean_Kkma[word][w_divided-1][0][-1], Toknized_Korean_Kkma[word][w_divided][0])

                if Toknized_Korean_Kkma[word][w_divided][1] == 'ETD' or Toknized_Korean_Kkma[word][w_divided][1] =='EFN':
                    if Toknized_Korean_Kkma[word][w_divided][0] == "ㄴ":
                        Toknized_Korean_Kkma[word][w_divided][0] == "한"
                    elif Toknized_Korean_Kkma[word][w_divided][0] == "ㄴ다":
                        Toknized_Korean_Kkma[word][w_divided][0] == "한다"



                vogulized_sentence += str(Toknized_Korean_Kkma[word][w_divided][0])

    vogulized_sentence += ' '


print(vogulized_sentence)
print(Korean_Text_b)

# nltk 같은 걸로 영어 품사에 따라 주격조사 은,는,이,가 지정하기
# ex) 맛있는 -> 딜리셔스는 -> 딜리셔스한
# 만약 영어 형용사 + 뒤에 은/는 이면 한으로 주격조사 바꾸기

번역 가능한 품사 단어 : 안녕하세요
헬로
번역 불가능한 품사 단어 : 하세
번역 가능한 품사 단어 : 만나
미트
번역 불가능한 품사 단어 : 서
번역 불가능한 품사 단어 : 웰컴
번역 불가능한 품사 단어 : 습니다
번역 불가능한 품사 단어 : .
번역 불가능한 품사 단어 : 굿데뉴윈
번역 불가능한 품사 단어 : 좋
번역 불가능한 품사 단어 : 스테이
번역 불가능한 품사 단어 : ㅂ시다
헬로하세 미트서 웰컴습니다. 굿데뉴윈좋 스테이ㅂ시다 
안녕하세요, 만나서 반갑습니다. 사이좋게 지냅시다
